# **Merge NDVI CSVs for Each Area**
This notebook merges NDVI CSV files for different areas and years into a single file per area.

🔹 **Google Drive Mounting**
🔹 **CSV File Merging**
🔹 **Saving the Results**

✅ **Run the cells below to execute the process!**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Import necessary libraries
import os
import glob
import pandas as pd

In [ ]:
# Set the folder containing the CSV files
folder_path = '/content/drive/MyDrive/earthengine/csv'

In [ ]:
# Get all CSV files
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))
print(f'Found {len(csv_files)} CSV files.')

In [ ]:
# Dictionary to store dataframes by area
area_data = {}

In [ ]:
# Process files
for file in csv_files:
    filename = os.path.basename(file)
    parts = filename.split('_')
    if len(parts) < 5:
        continue  # Skip files that don't match expected pattern
    
    year = parts[2]  # Extract year
    area = parts[4]  # Extract area number
    
    df = pd.read_csv(file)
    df = df.rename(columns={'y2013': f'y{year}'})  # Rename year column
    
    if area not in area_data:
        area_data[area] = df[['pixel_id', 'x_coord', 'y_coord', f'y{year}']]
    else:
        area_data[area] = area_data[area].merge(df[['pixel_id', f'y{year}']], on='pixel_id', how='outer')

In [ ]:
# Save merged data
output_folder = os.path.join(folder_path, 'merged')
os.makedirs(output_folder, exist_ok=True)

for area, df in area_data.items():
    output_file = os.path.join(output_folder, f'NDVI_Merged_Area_{area}.csv')
    df.to_csv(output_file, index=False)
    print(f'Saved: {output_file}')

print("Merging complete!")